In [97]:
import numpy as np
import torch
import torch.nn.functional as F

In [119]:
x = np.linspace(0,1,10, dtype=np.float)
t = np.linspace(0,1,10, dtype=np.float)
x, t = np.meshgrid(x, t) 
x = x.flatten()
t = t.flatten()
X = np.concatenate((x.reshape(-1,1), t.reshape(-1, 1)), axis=1)

N, D_in = X.shape
H = 100
D_out = 1

In [141]:
v1, v2 = x[5]
print(v1, v2)
v1, v2 = x[5].T
print(v1, v2)

tensor(0.5556) tensor(0.)
tensor(0.5556) tensor(0.)


In [146]:
def g_bc(val):
    x, t = val.T
    return (1 - t) * torch.sin(np.pi * x)

def g_nn(val, N):
    x, t = val.T
    return x * (1 - x) * t * N

def g_trial(val, N):
    x, t = val.T
    return g_bc(x, t) + g_nn(x, t, N)

def ode_loss():
    # Need to compute Hessian and Jacobian using pytorch
    pass


In [147]:
dtype = torch.float
#device = torch.device("cuda:0")
device = torch.device("cpu")

x = torch.from_numpy(X)
x = x.to(dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-7
for t in range(500):
    N_output = F.relu(x.mm(w1)).mm(w2)

    # Compute and print loss
    loss = ode_loss(x, N_output)
    print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

TypeError: ode_loss() takes 0 positional arguments but 2 were given